In [1]:
from kafka import KafkaProducer
import requests
from json import dumps
import time
from kafka import KafkaConsumer
from json import loads
import pandas as pd

In [2]:
consumer = KafkaConsumer(
    'Product',
     bootstrap_servers=['127.0.0.1:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Upload Product to HDFS").setMaster("spark://25.15.27.228:7077")
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Upload Product to HDFS').getOrCreate()

In [5]:
df = []
iter = 0
for message in consumer:
    iter +=1
    message = message.value
    df.append(message)
    if iter %1000 ==0:
        iter = 0
        print(len(df))
    if len(df) > 5000:
        print('Upload Product to HDFS')
        try:
            pandas_df = pd.DataFrame(df)
            df = []
            data = spark.createDataFrame(pandas_df.astype(str))
            data.coalesce(1).write.mode('append').parquet('hdfs://cris:9000/shopee/Product/')
        except:
            print('Error when upload data to HDFS')

1000


In [ ]:
len(df)

In [7]:
pandas_df = pd.DataFrame(df)
data = spark.createDataFrame(pandas_df.astype(str))
data.coalesce(1).write.mode('append').csv('hdfs://cris:9000/shopee/Item/')

In [68]:
# tmp = spark.read.csv('hdfs://cris:9000/shopee/Item/part-00000-ad86ee4c-be73-4872-9f2f-88c1a8cae58f-c000.csv')